## 计算图的概念

- 概念

    计算图是描述计算的有向无环图
    
- 组成

    计算图主要由节点（Node）和边（Edge）两个元素组成
    
**结点**
- 表示数据张量

**边**
- 表示运算，如加减乘除卷积等操作

**例**


$$y=(w+x)*(w+1)$$

- 分解

$$a=w+x$$

$$b=w+1$$

$$y=a*b$$


- 则对应计算图为：

![](./img/jisuan.png)

- 好处

    方便计算与求导
    
    
![](./img/daoshu.png)

In [1]:
import torch

In [3]:
# ===============================  exmaple 1 ===============================
# 通过pytorch构建计算图

##初始化x和w，注意需要计算导数
x = torch.tensor([2.], requires_grad=True)
w = torch.tensor([1.], requires_grad=True)

##构建计算图
one = torch.tensor([1.])
a = torch.add(w, x)
b = torch.add(w, one)
y = torch.mul(a, b)

**张量的叶子结点is_leaf**
    

- 叶子结点的概念

    用户创建的张量为结点张量，如上计算图中的$x$和$w$,is_leaf为True时，表示该张量为结点张量
    
    当反向传播求导数时，只会保存叶子结点的导数（$x$和$w$），非叶子结点（如$a和b$）不会被保存导数

In [4]:
# ===============================  exmaple 2 ===============================
# 通过pytorch构建反向传播求导数，默认不保存中间变量的梯度

##反向传播计算
y.backward()
print("x_isleaf:{}\tw_isleaf:{}\ta_isleaf:{}\tb_isleaf:{}".format(
    x.is_leaf, w.is_leaf, a.is_leaf, b.is_leaf))
print("x_grad:{}\tw_grad:{}\ta_grad:{}\tb_grad:{}".format(
    x.grad, w.grad, a.grad, b.grad))

x_isleaf:True	w_isleaf:True	a_isleaf:False	b_isleaf:False
x_grad:tensor([2.])	w_grad:tensor([5.])	a_grad:None	b_grad:None


- 注解

    默认情况下，只有叶子结点的张量导数才会被保存，否则，中间导数不会被保存
    
    如果想保存中间变量的梯度，可以使用retain_grad()

In [5]:
# ===============================  exmaple 3 ===============================
# 通过pytorch构建反向传播求导数，保存中间变量的梯度

x = torch.tensor([2.], requires_grad=True)
w = torch.tensor([1.], requires_grad=True)

a = torch.add(w, x)
a.retain_grad()  ##采用retain_grad()保存中间变量的梯度
b = torch.add(w, 1)
b.retain_grad()

y = torch.mul(a, b)

y.backward()

print("x_isleaf:{}\tw_isleaf:{}\ta_isleaf:{}\tb_isleaf:{}".format(
    x.is_leaf, w.is_leaf, a.is_leaf, b.is_leaf))
print("x_grad:{}\tw_grad:{}\ta_grad:{}\tb_grad:{}".format(
    x.grad, w.grad, a.grad, b.grad))

x_isleaf:True	w_isleaf:True	a_isleaf:False	b_isleaf:False
x_grad:tensor([2.])	w_grad:tensor([5.])	a_grad:tensor([2.])	b_grad:tensor([3.])


- grad_fn

    tensor的grad_fn表示这个tensor的生成方式

In [8]:
# 查看 grad_fn
print("grad_fn:\n", w.grad_fn, x.grad_fn, a.grad_fn, b.grad_fn, y.grad_fn)
##注意由于w和x是用户创建的，因此没有运算方式

grad_fn:
 None None <AddBackward0 object at 0x00000000064A3860> <AddBackward0 object at 0x00000000064A36A0> <MulBackward0 object at 0x00000000064A3320>


## 动态计算图

![](./img/dongtai.png)